In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from preprocessing_weather import WEATHER
from preprocessing_pedestrians import PEDESTRIANS
from preprocessing_permits import PERMITS

          Location       Date  Temperature mean  Temperature max  \
27125   St. Gallen 2019-01-08               0.9              1.9   
27130   St. Gallen 2019-01-10              -3.2             -2.5   
27135   St. Gallen 2019-01-18              -2.8              0.7   
27140   St. Gallen 2019-01-27               2.5              5.0   
27145   St. Gallen 2019-02-01               1.2              4.8   
...            ...        ...               ...              ...   
167138  St. Gallen 2024-10-29              10.8             13.6   
167139  St. Gallen 2024-10-30               8.5             11.2   
167140  St. Gallen 2024-11-07               6.5              7.6   
167141  St. Gallen 2024-11-10               3.9              5.6   
167142  St. Gallen 2024-11-22              -2.5             -1.0   

        Temperature min  Precipitation in mm  Snow amount in cm  
27125              -0.8                  4.2               35.0  
27130              -3.8                 17.3       

In [4]:
print(PERMITS.head())

                                          Objektname BaustKanton  \
0                      Neubau Primarschule Riethüsli          SG   
1  Teilabbruch und Erweiterung der Kantonsschule ...          SG   
3  Erweiterung/Erneuerung Regionalgefängnis mit P...          SG   
4  Neubau Pflegezentrum Schachen mit Tiefgarage u...          SG   
5    Neubau Gewerbezentrum/Parkunique mit Tiefgarage          SG   

  BaustadiumDatum          BaustadiumAlt BaustadiumDatumAlt      Baukosten  \
0      2022-06-03  Baugesuch eingereicht         2021-11-22  48.00 Mio CHF   
1      2021-11-30  Baugesuch eingereicht         2015-10-02  49.00 Mio CHF   
3      2020-02-05  Baugesuch eingereicht         2019-06-17  83.00 Mio CHF   
4      2023-07-05  Baugesuch eingereicht         2019-11-21  46.70 Mio CHF   
5      2024-11-15                Projekt         2017-01-06  25.00 Mio CHF   

   Baubeginn  Bauende  
0          1        1  
1          1        1  
3          1        1  
4          1        1  
5 

In [5]:
print(PEDESTRIANS.head())

         Date  Day  Workday  Total Pedestrians
0  2024-03-24    7        0              420.0
8  2024-03-25    1        1             1766.0
19 2024-03-26    2        1             1701.0
27 2024-03-28    4        1             1669.0
33 2024-03-29    5        1              665.0


In [6]:
print(WEATHER.head())

         Location       Date  Temperature mean  Temperature max  \
27125  St. Gallen 2019-01-08               0.9              1.9   
27130  St. Gallen 2019-01-10              -3.2             -2.5   
27135  St. Gallen 2019-01-18              -2.8              0.7   
27140  St. Gallen 2019-01-27               2.5              5.0   
27145  St. Gallen 2019-02-01               1.2              4.8   

       Temperature min  Precipitation in mm  Snow amount in cm  
27125             -0.8                  4.2               35.0  
27130             -3.8                 17.3               39.0  
27135             -7.0                  0.0               32.0  
27140             -0.9                  5.3               27.0  
27145             -1.3                  0.2               37.0  


In [7]:
# Merge the datasets
# Start with Pedestrians and Weather datasets and merge them on the Date column, performing an inner join to only keep values that are in both datasets
merged_features = pd.merge(PEDESTRIANS, WEATHER, on='Date', how='inner')
print(merged_features)

           Date  Day  Workday  Total Pedestrians    Location  \
0    2024-03-24    7        0              420.0  St. Gallen   
1    2024-03-25    1        1             1766.0  St. Gallen   
2    2024-03-26    2        1             1701.0  St. Gallen   
3    2024-03-28    4        1             1669.0  St. Gallen   
4    2024-03-29    5        1              665.0  St. Gallen   
...         ...  ...      ...                ...         ...   
2044 2024-10-23    3        1             1221.0  St. Gallen   
2045 2024-10-24    4        1             1335.0  St. Gallen   
2046 2024-10-25    5        1             1404.0  St. Gallen   
2047 2024-10-26    6        0             1600.0  St. Gallen   
2048 2024-10-12    6        0             2309.0  St. Gallen   

      Temperature mean  Temperature max  Temperature min  Precipitation in mm  \
0                  1.7              3.8             -0.2                  5.2   
1                  4.7              9.3              0.4             

In [8]:
print(PERMITS['BaustadiumDatum'].nunique())
print(2049+1761)


339
3810


In [9]:
print(PERMITS.shape)

(1761, 8)


In [ ]:
# Now merge the Permits dataset with the merged dataset from the previous step, this time performing a left merge to make sure we keep all the values from the already merged dataset
merged = pd.merge(merged_features, PERMITS, left_on='Date', right_on='BaustadiumDatum', how='left')
print(merged.sort_values(by='Date'))
print(merged.shape)

(1689, 18)


In [ ]:
# We notice that there are some NaN values in the BaustadiumDatum column, which means that there are some dates in the Pedestrians and Weather dataset that are not in the Permits dataset
# Which makes sense since there aren't new construction permits happening every day
# Find NaN values in the merged_features dataframe and print them
nan_values = merged.isna().sum()
#print(nan_values)
# Check for duplicate rows in the merged dataframe
duplicates = merged[merged.duplicated()]
print(merged[merged["Date"] == "2024-10-08"])
print(merged.loc[1515])

Date                       2024-10-08 00:00:00
Day                                          2
Workday                                      1
Total Pedestrians                       1541.0
Location                            St. Gallen
Temperature mean                          12.8
Temperature max                           15.5
Temperature min                           10.9
Precipitation in mm                       10.1
Snow amount in cm                          0.0
Objektname             Abbruch Einfamilienhaus
BaustKanton                                 SG
BaustadiumDatum            2024-10-08 00:00:00
BaustadiumAlt            Baugesuch eingereicht
BaustadiumDatumAlt         2024-02-05 00:00:00
Baukosten                         0.03 Mio CHF
Baubeginn                                    0
Bauende                                      0
Name: 1515, dtype: object


In [12]:
# Create a mask for NaN values
nan_mask = merged.isna()

# Filter and print the rows that contain NaN values
nan_rows = merged[nan_mask.any(axis=1)]  # Select rows with NaN values
print("Rows with NaN values:")
print(nan_rows)

Rows with NaN values:
           Date  Day  Workday  Total Pedestrians    Location  \
0    2024-03-24    7        0              420.0  St. Gallen   
1    2024-03-25    1        1             1766.0  St. Gallen   
18   2024-03-30    6        0             2201.0  St. Gallen   
19   2024-03-31    7        0              569.0  St. Gallen   
20   2024-04-01    1        1              455.0  St. Gallen   
...         ...  ...      ...                ...         ...   
3376 2024-10-21    1        1             1302.0  St. Gallen   
3377 2024-09-28    6        0             1743.0  St. Gallen   
3378 2024-09-27    5        1             1810.0  St. Gallen   
3409 2024-10-26    6        0             1600.0  St. Gallen   
3410 2024-10-12    6        0             2309.0  St. Gallen   

      Temperature mean  Temperature max  Temperature min  Precipitation in mm  \
0                  1.7              3.8             -0.2                  5.2   
1                  4.7              9.3        

In [13]:
# Filter and print the columns that contain NaN values
nan_columns = merged.loc[:, nan_mask.any(axis=0)]  # Select columns with NaN values
print("\nColumns with NaN values:")
print(nan_columns)


Columns with NaN values:
                                  Objektname BaustKanton BaustadiumDatum  \
0                                        NaN         NaN             NaT   
1                                        NaN         NaN             NaT   
2              Abbruch Doppeleinfamilienhaus          SG      2024-03-26   
3                           Abbruch Wekhalle          SG      2024-03-26   
4                           Neubau Werkhalle          SG      2024-03-26   
...                                      ...         ...             ...   
3406   Neubau 2 Doppelhäuser und  Tiefgarage          SG      2024-10-25   
3407    Umbau und Sanierung Mehrfamilienhaus          SG      2024-10-25   
3408  Neubau Mehrfamilienhaus mit Tiefgarage          SG      2024-10-25   
3409                                     NaN         NaN             NaT   
3410                                     NaN         NaN             NaT   

              BaustadiumAlt BaustadiumDatumAlt          Bauko

In [35]:
# Group by 'Date' and count how many rows in df2 have the same date
count_per_date = PERMITS.groupby('BaustadiumDatum').size()

# Map the count to the merged DataFrame using the 'Date' column
merged['Amount of Permits'] = merged['Date'].map(count_per_date).fillna(0).astype(int)

# Display the result
print(merged[merged['Amount of Permits'] == 0])

           Date  Day  Workday  Total Pedestrians    Location  \
0    2024-03-24    7        0              420.0  St. Gallen   
1    2024-03-25    1        1             1766.0  St. Gallen   
18   2024-03-30    6        0             2201.0  St. Gallen   
19   2024-03-31    7        0              569.0  St. Gallen   
20   2024-04-01    1        1              455.0  St. Gallen   
...         ...  ...      ...                ...         ...   
3376 2024-10-21    1        1             1302.0  St. Gallen   
3377 2024-09-28    6        0             1743.0  St. Gallen   
3378 2024-09-27    5        1             1810.0  St. Gallen   
3409 2024-10-26    6        0             1600.0  St. Gallen   
3410 2024-10-12    6        0             2309.0  St. Gallen   

      Temperature mean  Temperature max  Temperature min  Precipitation in mm  \
0                  1.7              3.8             -0.2                  5.2   
1                  4.7              9.3              0.4             

In [18]:
print("Keys in merged_features1:", merged_features['Date'].unique())
print("Keys in Permits:", PERMITS['BaustadiumDatum'].unique())

Keys in merged_features1: <DatetimeArray>
['2024-03-24 00:00:00', '2024-03-25 00:00:00', '2024-03-26 00:00:00',
 '2024-03-28 00:00:00', '2024-03-29 00:00:00', '2024-03-30 00:00:00',
 '2024-03-31 00:00:00', '2024-04-01 00:00:00', '2024-04-02 00:00:00',
 '2024-04-03 00:00:00',
 ...
 '2024-10-05 00:00:00', '2024-10-21 00:00:00', '2024-09-28 00:00:00',
 '2024-09-27 00:00:00', '2024-10-22 00:00:00', '2024-10-23 00:00:00',
 '2024-10-24 00:00:00', '2024-10-25 00:00:00', '2024-10-26 00:00:00',
 '2024-10-12 00:00:00']
Length: 2049, dtype: datetime64[ns]
Keys in Permits: <DatetimeArray>
['2022-06-03 00:00:00', '2021-11-30 00:00:00', '2020-02-05 00:00:00',
 '2023-07-05 00:00:00', '2024-11-15 00:00:00', '2024-06-27 00:00:00',
 '2024-08-28 00:00:00', '2023-10-19 00:00:00', '2024-04-08 00:00:00',
 '2024-10-17 00:00:00',
 ...
 '2022-04-07 00:00:00', '2021-11-04 00:00:00', '2024-08-06 00:00:00',
 '2023-06-14 00:00:00', '2022-04-27 00:00:00', '2024-11-27 00:00:00',
 '2022-12-22 00:00:00', '2023-09-19 0

In [19]:
# Count occurrences of each key in df1
merged_features_counts = merged_features.groupby('Date').size()
print("Occurrences of each key in merged_features1:")
print(merged_features_counts)

# Count occurrences of each key in df2
PERMITS_counts = PERMITS.groupby('BaustadiumDatum').size()
print("\nOccurrences of each key in PERMITS:")
print(PERMITS_counts)

Occurrences of each key in merged_features1:
Date
2019-03-08    1
2019-03-11    1
2019-03-12    1
2019-03-13    1
2019-03-14    1
             ..
2024-11-03    1
2024-11-04    1
2024-11-05    1
2024-11-06    1
2024-11-07    1
Length: 2049, dtype: int64

Occurrences of each key in PERMITS:
BaustadiumDatum
2020-02-05     1
2021-03-20     1
2021-11-04     1
2021-11-05     1
2021-11-28     1
              ..
2024-11-18     1
2024-11-19    11
2024-11-20    11
2024-11-21    11
2024-11-27     1
Length: 339, dtype: int64


In [20]:
# Merge with an indicator to see how many matches there are
potential_merge = merged_features.merge(PERMITS, left_on='Date', right_on='BaustadiumDatum', how='inner', indicator=True)

# Group by 'Date' to count the number of matches
merge_counts = potential_merge.groupby('Date').size()
print("\nNumber of rows that would result from merging:")
print(merge_counts)


Number of rows that would result from merging:
Date
2020-02-05    1
2021-03-20    1
2021-11-04    1
2021-11-05    1
2021-11-28    1
             ..
2024-10-30    1
2024-11-01    2
2024-11-04    1
2024-11-05    4
2024-11-06    2
Length: 327, dtype: int64


In [22]:
# Find rows with NaN values
nan_rows = merged[merged.isna().any(axis=1)]

# Get columns that contain NaN values
nan_columns = merged.columns[merged.isna().any()]

# Display rows and only the columns with NaN values
print("Rows and columns with NaN values:")
print(nan_rows[nan_columns])

Rows and columns with NaN values:
     Objektname BaustKanton BaustadiumDatum BaustadiumAlt BaustadiumDatumAlt  \
0           NaN         NaN             NaT           NaN                NaT   
1           NaN         NaN             NaT           NaN                NaT   
18          NaN         NaN             NaT           NaN                NaT   
19          NaN         NaN             NaT           NaN                NaT   
20          NaN         NaN             NaT           NaN                NaT   
...         ...         ...             ...           ...                ...   
3376        NaN         NaN             NaT           NaN                NaT   
3377        NaN         NaN             NaT           NaN                NaT   
3378        NaN         NaN             NaT           NaN                NaT   
3409        NaN         NaN             NaT           NaN                NaT   
3410        NaN         NaN             NaT           NaN                NaT   

     

In [ ]:
# Because there are duplicates we will introduce a new row which has the amount of permits granted that day.

Duplicate rows:
           Date  Day  Workday  Total Pedestrians    Location  \
778  2024-01-12    5        1             1376.0  St. Gallen   
1000 2023-12-08    5        1             2009.0  St. Gallen   
1037 2023-12-14    4        1             2170.0  St. Gallen   
1518 2024-10-08    2        1             1541.0  St. Gallen   
1519 2024-10-08    2        1             1541.0  St. Gallen   
1521 2024-10-08    2        1             1541.0  St. Gallen   

      Temperature mean  Temperature max  Temperature min  Precipitation in mm  \
778               -4.8             -4.0             -5.8                  0.0   
1000               2.9              5.0             -1.7                  3.9   
1037               3.3              4.0              1.3                  8.9   
1518              12.8             15.5             10.9                 10.1   
1519              12.8             15.5             10.9                 10.1   
1521              12.8             15.5          